<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GS-for-NB" data-toc-modified-id="GS-for-NB-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GS for NB</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-SVM" data-toc-modified-id="GS-for-SVM-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>GS for SVM</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-LR" data-toc-modified-id="GS-for-LR-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>GS for LR</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-RF" data-toc-modified-id="GS-for-RF-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>GS for RF</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li></ul></div>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Text-Classification/code")
!pip install pyLDAvis
!pip install gensim
!pip install pandas==1.3.0
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [17]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
nb_name = "svm_nb_lr_RF_baseline"

In [3]:
import numpy as np 
import pandas as pd
import random

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.pipeline import Pipeline

from classification_utils import *
from clustering_utils import *
from eda_utils import *
from nn_utils_keras import *
from feature_engineering_utils import *
from data_utils import *
setup_seed_ml(2021)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:
train, test = load_data()
# train, upsampling_info = upsampling_train(train)

train_text, train_label = train_augmentation(train, select_comb=[['text'], ['reply', 'reference_one']])
# train_text, train_label = train_augmentation(train, select_comb=[['text']])
test_text, test_label = test['text'], test['label']

# test_text = test_text.apply(lambda x: normal_string(x))
# train_text = train_text.apply(lambda x: normal_string(x))


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']
combination 1 train:  ['text']
combination 2 train:  ['reply', 'reference_one']


In [5]:
####################################
### label mapper
####################################
labels = sorted(train_label.unique())
label_mapper = dict(zip(labels, range(len(labels))))
train_label = train_label.map(label_mapper)
test_label = test_label.map(label_mapper)
y_train = train_label.values
y_test = test_label.values

print(train_text.shape)
print(test_text.shape)
print(train_label.shape)
print(test_label.shape)

(22166,)
(7761,)
(22166,)
(7761,)


# GS for NB

In [21]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#               'clf__alpha': [0.6, 0.8, 1]}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"NB_cv_result_{nb_name}.xlsx")
# set(y_test)

## Best params

In [7]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, stop_words=True, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# tfidf_vectorizer(train_text, test_text, min_df=2, max_df=100)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500) # not allow negative 
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = MultinomialNB(alpha=0.2)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
# cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
# pred_proba = clf.predict_proba(X_test)
# roc_auc(y_test, pred_proba)
# cv

num of words: 48692
X_train.shape (22166, 48692)
X_test.shape (7761, 48692)
classification_report:
f1:  {'micro avg': 0.7837907486148692, 'macro avg': 0.7873107453143067} 

                          precision  recall  f1-score  support
alt.atheism                   0.818   0.774     0.795    319.0
comp.graphics                 0.712   0.763     0.737    389.0
comp.os.ms-windows.misc       0.780   0.637     0.701    394.0
comp.sys.ibm.pc.hardware      0.644   0.791     0.710    392.0
comp.sys.mac.hardware         0.856   0.834     0.845    385.0
comp.windows.x                0.842   0.825     0.834    395.0
misc.forsale                  0.812   0.844     0.828    390.0
rec.autos                     0.913   0.904     0.908    395.0
rec.motorcycles               0.934   0.955     0.944    398.0
rec.sport.baseball            0.458   0.965     0.621    397.0
rec.sport.hockey              1.000   0.375     0.545    827.0
sci.crypt                     0.876   0.924     0.899    396.0
sci.elec

# GS for SVM

In [23]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#              'clf__penalty':['l2'], 'clf__C':[1,2,3]}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"SVC_cv_result_{nb_name}.xlsx")

## Best params

In [24]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, stop_words=False, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=3, dual=True)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
cv

num of words: 38308
X_train.shape (11083, 38308)
X_test.shape (7761, 38308)
classification_report:
f1:  {'micro avg': 0.8514366705321479, 'macro avg': 0.8403710771549164} 

                          precision  recall  f1-score  support
alt.atheism                   0.841   0.793     0.816    319.0
comp.graphics                 0.747   0.797     0.771    389.0
comp.os.ms-windows.misc       0.787   0.734     0.760    394.0
comp.sys.ibm.pc.hardware      0.743   0.730     0.736    392.0
comp.sys.mac.hardware         0.814   0.873     0.842    385.0
comp.windows.x                0.876   0.803     0.838    395.0
misc.forsale                  0.852   0.903     0.877    390.0
rec.autos                     0.909   0.911     0.910    395.0
rec.motorcycles               0.957   0.957     0.957    398.0
rec.sport.baseball            0.814   0.970     0.885    397.0
rec.sport.hockey              0.992   0.886     0.936    827.0
sci.crypt                     0.931   0.922     0.926    396.0
sci.elec

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,3.325843,0.403172,0.876287,0.999773,0.874414,0.999759,0.871787,0.999772,0.874414,0.999759
1,3.367836,0.392958,0.920398,0.999547,0.915193,0.999519,0.917482,0.999547,0.915193,0.999519
2,3.336643,0.299839,0.918826,0.999774,0.915554,0.999759,0.912954,0.999775,0.915554,0.999759
3,3.404069,0.379425,0.885260,0.999661,0.884477,0.999639,0.880854,0.999661,0.884477,0.999639


In [16]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, dense=False, stop_words=True, binary=True, min_df=1)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=3.0, dual=True)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
# cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
# cv

num of words: 130193
X_train.shape (22166, 130193)
X_test.shape (7761, 130193)
classification_report:
f1:  {'micro avg': 0.847571189279732, 'macro avg': 0.8348514977956949} 

                          precision  recall  f1-score  support
alt.atheism                   0.810   0.746     0.777    319.0
comp.graphics                 0.764   0.815     0.789    389.0
comp.os.ms-windows.misc       0.784   0.736     0.759    394.0
comp.sys.ibm.pc.hardware      0.719   0.742     0.730    392.0
comp.sys.mac.hardware         0.835   0.852     0.843    385.0
comp.windows.x                0.868   0.813     0.839    395.0
misc.forsale                  0.819   0.895     0.855    390.0
rec.autos                     0.923   0.916     0.920    395.0
rec.motorcycles               0.945   0.955     0.950    398.0
rec.sport.baseball            0.841   0.962     0.898    397.0
rec.sport.hockey              0.991   0.900     0.943    827.0
sci.crypt                     0.914   0.934     0.924    396.0
sci.el

# GS for LR

In [ ]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr', max_iter=1000))])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#              'clf__penalty':['l2'], 'clf__C':[1,2,3], 'clf__solver': ['liblinear', 'newton-cg', 'sag', 'saga' , 'lbfgs']}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"LR_1_cv_result_{nb_name}.xlsx")

In [14]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr'))])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#              'clf__penalty':['elasticnet'], 'clf__C':[1,2,3], 'clf__l1_ratio':[0.2, 0.5,0.8],'clf__solver': ['saga']}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"LR_2_cv_result_{nb_name}.xlsx")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


## Best params

In [15]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LogisticRegression(penalty="l2", multi_class='ovr', max_iter=1000)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
# cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
# pred_proba = clf.predict_proba(X_test)
# roc_auc(y_test, pred_proba)
# cv

num of words: 130526
X_train.shape (22166, 130526)
X_test.shape (7761, 130526)
classification_report:
f1:  {'micro avg': 0.8087875273804922, 'macro avg': 0.8070677974746536} 

                          precision  recall  f1-score  support
alt.atheism                   0.777   0.755     0.766    319.0
comp.graphics                 0.693   0.802     0.744    389.0
comp.os.ms-windows.misc       0.794   0.764     0.779    394.0
comp.sys.ibm.pc.hardware      0.773   0.763     0.768    392.0
comp.sys.mac.hardware         0.843   0.868     0.855    385.0
comp.windows.x                0.870   0.810     0.839    395.0
misc.forsale                  0.811   0.910     0.857    390.0
rec.autos                     0.910   0.922     0.916    395.0
rec.motorcycles               0.967   0.955     0.961    398.0
rec.sport.baseball            0.525   0.970     0.681    397.0
rec.sport.hockey              1.000   0.534     0.697    827.0
sci.crypt                     0.938   0.909     0.923    396.0
sci.e

# GS for RF

In [ ]:
from xgboost import XGBClassifier

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), 
                     ('clf', XGBClassifier(objective='multi:softmax', n_estimators=500, num_class=20, learning_rate=0.075, \
                                           colsample_bytree=0.7, subsample=0.8, eval_metric='merror'))])

parameters = {
    'tfidf__min_df': [1, 3], 'tfidf__stop_words': ['english'], 'tfidf__use_idf': [True], 'tfidf__binary': [True],
    #'svd__n_components': (250, 500, 750),
    #'clf__n_estimators': (250, 500, 750),
    'clf__max_depth': (4, 6, 8),
    'clf__min_child_weight': (1, 5, 10),
    #'clf__alpha': (0.00001, 0.000001),
    #'clf__penalty': ('l2', 'elasticnet'),
    #'clf__max_iter': (10, 50, 80),
}

gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"RF_1_cv_result_{nb_name}.xlsx")

## Best params

In [17]:
from xgboost import XGBClassifier

In [18]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(
    train_text, test_text, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500) # not allow negative
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)

# param = {'max_depth': 4, 'objective': 'multi:softmax', 'clf__min_child_weight': 1,
#          'booster':'gblinear', # linear model
#          'alpha': 0.0001,  # alpha is the L1 regularizer
#          'lambda': 1} # lambda is the L2 regularizer
# param['num_class'] = len(labels)
# param['eta'] = 0.1 # # scale weight of positive examples
# param['eval_metric'] = ['auc', 'ams@0']


clf = XGBClassifier(objective='multi:softmax', max_depth=4, min_child_weight=1, n_estimators=500, num_class=20, learning_rate=0.075,
                    colsample_bytree=0.7, subsample=0.8, eval_metric='merror')
# clf =  xgb.XGBClassifier(param)

clf.fit(X_train, y_train)
pred = clf.predict(X_test)

num of words: 49023
X_train.shape (22166, 49023)
X_test.shape (7761, 49023)


In [19]:
classification_report = evaluation_report(y_test, pred, labels=labels)
# cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
# pred_proba = clf.predict_proba(X_test)
# roc_auc(y_test, pred_proba)
# cv

classification_report:
f1:  {'micro avg': 0.847571189279732, 'macro avg': 0.8348514977956949} 

                          precision  recall  f1-score  support
alt.atheism                   0.810   0.746     0.777    319.0
comp.graphics                 0.764   0.815     0.789    389.0
comp.os.ms-windows.misc       0.784   0.736     0.759    394.0
comp.sys.ibm.pc.hardware      0.719   0.742     0.730    392.0
comp.sys.mac.hardware         0.835   0.852     0.843    385.0
comp.windows.x                0.868   0.813     0.839    395.0
misc.forsale                  0.819   0.895     0.855    390.0
rec.autos                     0.923   0.916     0.920    395.0
rec.motorcycles               0.945   0.955     0.950    398.0
rec.sport.baseball            0.841   0.962     0.898    397.0
rec.sport.hockey              0.991   0.900     0.943    827.0
sci.crypt                     0.914   0.934     0.924    396.0
sci.electronics               0.814   0.725     0.767    393.0
sci.med               

In [ ]:
# clf.save_model("xgb.bin")
# clf.load_model("xgb.bin")
xgb.plot_importance(clf)
xgb.to_graphviz(clf, num_trees=2)